In [19]:

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI
import ollama

In [20]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [21]:
# ed = Website("https://edwarddonner.com")
# print(ed.title)
# print(ed.text)

In [22]:
!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling dde5aa3fc5ff... 100% ▕████████████████▏ 2.0 GB                         
pulling 966de95ca8a6... 100% ▕████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da... 100% ▕████████████████▏ 7.7 KB                         
pulling a70ff7e570d9... 100% ▕████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5... 100% ▕████████████████▏   96 B                         
pulling 34bb5ab01051... 100% ▕████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


## Helper Functions

In [23]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [24]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [25]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

## Payload info

In [26]:
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-type": "application/json"}
MODEL = "llama3.2"

## First approach

In [27]:

def summarize_requests(url):
    website = Website(url)
    # print(payload)
    payload = {
        "model": MODEL,
        "messages": messages_for(website),
        "stream": False
    }
    response = requests.post(OLLAMA_API, headers=HEADERS, json=payload)
    if response.status_code != 200:
        raise Exception(f"Error: {response.status_code} - {response.text}")
    return response.json()["message"]["content"]

In [28]:
def display_summary_requests(url):
    summary = summarize_requests(url)
    display(Markdown(summary))

In [29]:
url = "https://edwarddonner.com"
display_summary_requests(url)

### Website Summary

#### Overview

The website is owned by Edward Donner, a tech enthusiast and co-founder of Nebula.io. The site appears to be a personal blog or portfolio showcasing his work in the field of artificial intelligence (AI) and language models.

#### Content

*   Edward introduces himself as a writer, DJ, and amateur electronic music producer.
*   He discusses his background as the founder and CEO of AI startup untapt, which was acquired in 2021.
*   The website highlights Nebula.io's work on applying AI to help people discover their potential and talent sourcing platform for recruiters.

#### News and Announcements

*   **April 21, 2025:** Release of "The Complete Agentic AI Engineering Course."
*   **January 23, 2025:** Announcement of an LLM Workshop – Hands-on with Agents.
*   **December 21, 2024:** Introduction to Mastering AI and LLM Engineering – Resources.
*   **November 13, 2024:** Welcome message for SuperDataScientists.

## Ollama Package approach

In [30]:
def summarize_package(url):
    website = Website(url)
    # print(payload)
    response = ollama.chat(model=MODEL, messages=messages_for(website))
    
    return response["message"]["content"]

In [31]:
def display_summary_package(url):
    summary = summarize_package(url)
    display(Markdown(summary))

In [32]:
url = "https://edwarddonner.com"
display_summary_requests(url)

**Summary of the Website**
==========================

*   The website belongs to Edward Donner, a co-founder and CTO of Nebula.io, an AI startup.
*   The website discusses various topics related to artificial intelligence (AI) and language models.
*   **Upcoming Events:**
    *   The Complete Agentic AI Engineering Course (April 21, 2025)
    *   LLM Workshop – Hands-on with Agents – resources (December 21, 2024)
    *   Welcome, SuperDataScientists! (November 13, 2024)
*   **Past Events:**
    *   Mastering AI and LLM Engineering – Resources
    *   LLM Workshop 
*   The website provides a personal touch, with Edward Donner sharing his interests in writing code, DJing, and amateur electronic music production.

## Through OpenAI Library

In [33]:
OLLAMA_API = "http://localhost:11434/v1" # Notice that the API doesnt have /chat in it
ollama_via_openai = OpenAI(base_url=OLLAMA_API, api_key='ollama')

In [34]:
def summarize_openai(url):
    website = Website(url)
    messages = messages_for(website)
    # print(messages)
    response = ollama_via_openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [35]:
def display_summary_openai(url):
    summary = summarize_openai(url)
    display(Markdown(summary))

In [36]:
url = "https://edwarddonner.com"
display_summary_openai(url)

**Summary**

### Edward Donner's Website

The website revolves around Ed Donner, a pioneer in the AI industry. He is the co-founder and CTO of Nebula.io, an organization applying AI to help people discover their potential.

#### News/Announcements:

* **The Complete Agentic AI Engineering Course** - Announced on April 21, 2025
* **LLM Workshop – Hands-on with Agents – resources** - Published on December 21, 2024
* **Mastering AI and LLM Engineering – Resources** - Promoted in November 2024

#### Website Content:

- Ed Donner introduces himself as a code enthusiast and AI pioneer with interests in DJing, amateur electronic music production, and Hacker News.
- He describes Nebula.io's mission to apply AI positively and provides an overview of his previous ventures, including the acquired AI startup untapt.
- The website features links to various resources, including the LLM Workshop, for enthusiasts interested in learning about Agents.

#### Navigation:

The navigation menu offers links to 'Connect Four', 'Outsmart' (an arena where LLMs compete), 'About', and other resources.